# Esercizio 4 — Analisi meteo (propedeutico)

**Obiettivi:**
- Capire come costruire un `DataFrame` con pandas;
- Calcolare statistiche (quantili, medie) e creare colonne derivate;
- Raggruppare e trasformare i dati con `groupby` e `transform`.

Segui le celle successive: ogni blocco di codice è preceduto da una breve spiegazione didattica.


## 1) Setup: import e creazione del `DataFrame`
Questa cella importa `pandas` e crea un `DataFrame` chiamato `df`.
- `Giorno`: giorno della settimana;
- `Temperatura`: temperatura registrata (°C);
- `Umidita`: umidità in %;
- `Pioggia`: 1 se ha piovuto, 0 altrimenti;
- `Vento`: velocità del vento (km/h o unità generiche).
Esegui la cella per vedere la tabella iniziale.


In [ ]:
import pandas as pd

df = pd.DataFrame({
    "Giorno": ["Lun", "Mar", "Mer", "Gio", "Ven", "Sab", "Dom"],
    "Temperatura": [18, 20, 21, 19, 23, 25, 22],
    "Umidita": [60, 55, 58, 63, 50, 45, 52],
    "Pioggia": [0, 1, 0, 1, 0, 0, 1],
    "Vento": [5, 12, 8, 20, 7, 6, 15]
})

df

,Giorno,Temperatura,Umidita,Pioggia,Vento
0,Lun,18,60,0,5
1,Mar,20,55,1,12
2,Mer,21,58,0,8
3,Gio,19,63,1,20
4,Ven,23,50,0,7
5,Sab,25,45,0,6
6,Dom,22,52,1,15


## 2) Quantile: individuare umidità 'alta'
Calcoliamo il 75° percentile (`q3`) della colonna `Umidita` per definire cosa consideriamo 'umidità alta'.
Questo è utile per creare una soglia basata sui dati piuttosto che su un valore arbitrario.


In [ ]:
dfumiditaq3 = df["Umidita"].quantile(0.75)
dfumiditaq3

np.float64(59.0)

## 3) Creare una colonna booleana `umido_alto`
Usiamo la soglia `dfumiditaq3` per segnare le righe con umidità maggiore del 75° percentile.
Questa colonna è utile per filtri e analisi successive.


In [ ]:
df["umido_alto"] = df["Umidita"] > dfumiditaq3

df

,Giorno,Temperatura,Umidita,Pioggia,Vento,umido_alto
0,Lun,18,60,0,5,True
1,Mar,20,55,1,12,False
2,Mer,21,58,0,8,False
3,Gio,19,63,1,20,True
4,Ven,23,50,0,7,False
5,Sab,25,45,0,6,False
6,Dom,22,52,1,15,False


## 4) Categorizzare il vento
Definiamo tre categorie per la velocità del vento: `Basso`, `Medio`, `Alto`
- `Alto`: `Vento` >= 15;
- `Medio`: 8 <= `Vento` < 15;
- `Basso`: `Vento` < 8.
Queste soglie sono esemplificative: in un progetto reale le definiresti in base al dominio.


In [ ]:
vento_alto = df["Vento"] >= 15
vento_medio = (df["Vento"] >= 8) & (df["Vento"] < 15)
vento_basso = df["Vento"] < 8

df.head()

,Giorno,Temperatura,Umidita,Pioggia,Vento,umido_alto
0,Lun,18,60,0,5,True
1,Mar,20,55,1,12,False
2,Mer,21,58,0,8,False
3,Gio,19,63,1,20,True
4,Ven,23,50,0,7,False


## 5) Assegnare la classe del vento in una nuova colonna
Usiamo `loc` per impostare valori condizionali sulla colonna `Vento_Class`.
Questo approccio imposta prima un valore di default e poi sovrascrive per le condizioni specifiche.


In [ ]:
df["Vento_Class"] = "Basso"
df.loc[vento_medio, "Vento_Class"] = "Medio"
df.loc[vento_alto, "Vento_Class"] = "Alto"

df

,Giorno,Temperatura,Umidita,Pioggia,Vento,umido_alto,Vento_Class
0,Lun,18,60,0,5,True,Basso
1,Mar,20,55,1,12,False,Medio
2,Mer,21,58,0,8,False,Medio
3,Gio,19,63,1,20,True,Alto
4,Ven,23,50,0,7,False,Basso
5,Sab,25,45,0,6,False,Basso
6,Dom,22,52,1,15,False,Alto


## 6) Confrontare medie per gruppi (Pioggia x Classe vento)
Usiamo `groupby` per calcolare la media di `Temperatura` e `Umidita` per ogni combinazione di `Pioggia` e `Vento_Class`.
Questo aiuta a capire come il vento e la pioggia influenzano temperatura/umidità.


In [ ]:
df.groupby(["Pioggia", "Vento_Class"])[["Temperatura", "Umidita"]].mean()

Temperatura    Umidita
Pioggia Vento_Class                        
0       Basso               22.0  51.666667
        Medio               21.0  58.000000
1       Alto                20.5  57.500000
        Medio               20.0  55.000000

## 7) Aggiungere una colonna con la media di umidità per gruppo
Con `transform('mean')` applichiamo la media del gruppo a ogni riga appartenente a quel gruppo: otteniamo così `Umidita_Vento`.
Questa tecnica è utile quando vuoi confrontare il valore della riga con la statistica del suo gruppo.


In [ ]:
df["Umidita_Vento"] = df.groupby(["Pioggia", "Vento_Class"])["Umidita"].transform("mean")
df

,Giorno,Temperatura,Umidita,Pioggia,Vento,umido_alto,Vento_Class,Umidita_Vento
0,Lun,18,60,0,5,True,Basso,51.666667
1,Mar,20,55,1,12,False,Medio,55.000000
2,Mer,21,58,0,8,False,Medio,58.000000
3,Gio,19,63,1,20,True,Alto,57.500000
4,Ven,23,50,0,7,False,Basso,51.666667
5,Sab,25,45,0,6,False,Basso,51.666667
6,Dom,22,52,1,15,False,Alto,57.500000


## Riepilogo funzioni e tecniche (Analisi meteo)
Spiegazione di ogni operazione rilevante usata per arricchire e analizzare il DataFrame.

### Creazione dati
- `pd.DataFrame({...})`: costruisce la tabella con colonne Giorno, Temperatura, Umidita, Pioggia, Vento.

### Calcolo di statistiche base
- `df['Temperatura'].mean()`, `df['Umidita'].quantile(0.75)` (se usato): metodi su `Series` per estrarre misure riassuntive.
- Quantili: valore sotto cui cade una certa percentuale dei dati (es. 0.75 = 75%).

### Colonne booleane
- `df['umido_alto'] = df['Umidita'] >= soglia`: True/False per classificare rapidamente.

### Classificazione vento
- Se presente una logica tipo: `df['Vento_Class'] = np.where(df['Vento'] < 10, 'Basso', ...)` oppure uso di `pd.cut`. Obiettivo: trasformare valori numerici in categorie esplicative.
  - `np.where(cond, a, b)`: vettoriale, restituisce `a` dove `cond` è True, altrimenti `b`.
  - `pd.cut(serie, bins, labels=...)`: segmenta intervalli numerici in classi.

### Operazioni vettoriali
- `df['Nuova'] = df['Umidita'] * df['Vento']` (esempio): sfrutta operazioni elemento‑per‑elemento senza cicli.

### GroupBy multi‑colonna
- `df.groupby(['Pioggia','Vento_Class'])['Umidita']`: crea gruppi incrociando pioggia (0/1) e classe del vento.

### `transform('mean')` su gruppi
- `.transform('mean')`: assegna a ogni riga la media dell'umidità del proprio gruppo (non riduce il numero di righe).
- Risultato inserito in `df['Umidita_Vento']`: colonna comparativa utile per vedere se la riga è sopra/sotto la media di quel contesto (pioggia + vento).

### Interpretazione della media di gruppo
- Se `Umidita` > `Umidita_Vento`: umidità sopra la media del gruppo.
- Se `Umidita` < `Umidita_Vento`: sotto la media.

### Ordinamento / ispezione (se usato)
- `df.sort_values('Umidita_Vento')`: per analizzare gruppi più umidi / secchi.

### Buone pratiche
- Preferisci `transform` per unire statistiche al DataFrame originale.
- Usa nomi espliciti (`*_Class`, `*_mean`, `*_flag`).
- Mantieni logiche di soglia (es. vento basso/medio/alto) in una funzione riutilizzabile se diventano complesse.

### Glossario rapido
- Boolean indexing: filtrare righe con maschere True/False.
- Feature engineering: creare colonne derivate per analisi più ricche.
- GroupBy: suddividere i dati in sottoinsiemi omogenei per calcolare statistiche.

Con questo riepilogo ogni trasformazione ha un contesto e uno scopo chiaro.
